# Gaussian Process Regression with Interval Arithmetic

One current goal of the CMGDB group is to take in input data, use a Gaussian Process (GP) to define a map with that data (the map is the mean of the process), use that map to define a multivalued map which tracks rigorous bounds on where the image of the map lands in a grid, and then run that multivalued map through the CMGDB program in order to determine the Morse graph.  Hopefully that Morse graph is consistant with the true dynamics with high probability; that is a theoretical question that will take some work.

However, at the moment, we are having a much more practical problem; the software used in order to get the rigorous multivalued map that we desire (PyInterval) does not play well with the software used in order to get the GP generated map (sklearn GaussianProcessRegressor).  This code is a first attempt at solving that problem.  This should be doable, because the output of the Gaussian Process regression (the mean function of the process) is an explicit (and relatively simple) formula: $$ \mu^* = K(X_*,X)(K(X,X)+\sigma^2 I)^{-1}\hat{y} $$
Letting $m$ be the number of training points and $n$ be the dimension of the input vectors, we have that $X \in R^{m\times n}$, $K(X,X), I \in R^{m \times m}$, and $K(X,X)_{i,j} = k(x^{(i)},x^{(j)})$ where $k$ is the kernal.  Also $\hat{y} \in R^{m \times q}$ is the training output and $\sigma^2 \in R$ is the variance of the noise in the process. Note that the GP treats each of the $q$ dimensions of $y$ as independent.  

Note that, for fixed input data, 
$$ Z:= (K(X,X)+\sigma^2 I)^{-1}\hat{y}$$ is just an $m\times q$ matrix (real-valued).  Then the output of the GP is actually a pretty simple function where, taking the view that $X_* = x_*$ (so we just write the output of each point separately, so that we can view this as a function), we get that
$$ \mu(x_*) = K(x_*,X)Z$$
where
$$K(x_*,X) = [k(x_*,x^{(1)}),\cdots,k(x_*,x^{(m)})].$$

Assuming that we are using the common squared exponential kernal, the entries of the covariance matrix $K$ involve only exponentials, powers, division, and a Euclidean norm (so subtraction, addition, square root).  All of these functions are implemented in PyInterval.

Our interest is in performing interval arithmetic with respect to the test data $X_*$, so we we basically need to implement two steps. 

First, we compute $Z$ for a given input data.  We will do this using Cholesky factorization, following algorithm 2.1 from "Gaussian Processes for Machine Learning".  Note that this algorithm is what the sklearn function uses as well.  Since this part of the equation does not depend on the test data $X_*$, we do not need to follow interval arithmetic here (which is lucky, as the matrix inversion would probably be ridiculously slow).  

The second step (and where we break from the sklearn code) is to solve for the vector $K(x_*,X)$ using interval arithmetic, and multiply that vector by $Z$.

I also want to note that this code is a somewhat limited product; it is not be nearly as robust as the sklearn function; in particular, it is limited to only the use of the squared exponential kernal. It also only tracks the mean of the process, and not the variance.  At this moment we also work only in 2d, which is because that was the dimension of the problem we were interested in.


In [1]:
# for matrix inversion
from scipy.linalg import cholesky, cho_solve

# for defining K(X,X)
from scipy.spatial.distance import cdist

# the classics
import numpy as np
# import matplotlib.pyplot as plt
# import math
# import itertools 

# for interval arithmetic
from interval import interval, imath, fpu

# comparing to prebuilt packages
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF


In [27]:
def Z_matrix(X,Y,tau,sig_sq = 1e-10):
    """
    Input: 
        X: an m by n matrix (training input)
        Y: an m-vector (training output)
        tau: parameter in the squared exponential kernal
        sig_sq: noise level of errors in GP model.  Corresponds to alpha in sklearn
            Note: even if zero noise is assumed, sig_sq should be set positive to avoid numerical errors in the matrix inversion.    
    Output:
        Z: an m-vector, Z = (K(X,X)+sig_sq*I)^(-1)Y
    """
    # finding m
    m = X.shape[0]
    
    # defining the covariance matrix
    dists = cdist(X / tau, X / tau, metric="sqeuclidean")
    K = np.exp(-0.5 * dists)
    
    # Defining matrix that must be inverted
    A = K + sig_sq*np.eye(m)
    
    # Alg. 2.1, page 19, line 2 -> L = cholesky(K + sigma^2 I)
    L = cholesky(A, lower=True, check_finite=False)
    
    # Alg 2.1, page 19, line 3 -> alpha = L^T \ (L \ Y)
    Z = cho_solve((L, True),Y,check_finite=False)
    
    return Z

# interval squared exponential kernal
def sq_exp_intvl(a,b,tau):
    '''
    Input:
        a: length q vector
        b: length q vector
        tau: parameter in square exponential kernal
    Output: 
        k: interval result of square exponential kernal on a,b with parameter tau
    '''
    # notice that squaring the norm cancels out the square root operation
    norm_sq_list = [(interval(a[i]) - interval(b[i]))**2 for i in range(len(a))]
    norm_sq = sum(norm_sq_list)
    denom = -2*interval(tau)**2
    return imath.exp(norm_sq/denom)

# Define interval GP Regression
def GP_reg_intvl(X_train, x_star, Z, tau):
    """
    This function combines with GP_input_setup in order to do GP regression
    Evaluates at points in R^n--single points, not collection
    This version does interval arithmetic
    
    Input:
        X_train: an m by q matrix (training input)
        x_star: a 1 by q matrix (where we evaluate the regression)
        Z: m-vector, output of GP_input_setup
        tau: parameter for squared exponential kernal
        
    Output:
        Y_star: a m by q matrix, output of Gaussian regression of x_star
    """
    # finding dimensions
    m = X_train.shape[0]
    
    # defining the covariance matrix K(X*,X)
    KX_starX = []
    for j in range(m):
        xj = X_train[j,:]
        KX_starX.append(sq_exp_intvl(x_star,xj,tau))
    
    # getting result of regression, Y_star
    # have to basically redefine linear algebra here by hand to implement interval arithmetic
    Y_star = []
    for j in range(Z.shape[1]): # for 2D case (like Leslie map) Z.shape[1] = 2
        yj_list = [0]*m
        for i in range(m):
            yj_list[i] = KX_starX[i]*Z[i,j]
        Y_star.append(sum(yj_list))
                    
    return Y_star

In [28]:
# comparison to sklearn:

# Define a Gaussian process
def GP(X_train, Y_train):
    # fit Gaussian Process with dataset X_train, Y_train
    kernel = RBF(length_scale = tau, length_scale_bounds = "fixed")
    gp = GaussianProcessRegressor(kernel=kernel)
    gp.fit(X_train, Y_train)
    return gp

# Load data from file
data = np.loadtxt('Leslie_500.dat')

# Get X and Y vectors
X = data[:, [0,1]]
Y = data[:, [2,3]]

# save training input points
X_train = X

# set length scale 
tau = 5

In [29]:
%%time

# Train a GP with the data above
gp = GP(X, Y)

# Use the GP to define a map f
# Notice that the GP takes lists as
# input and output, so we need to
# add the [] below
def f(X):
    return gp.predict([X])[0]

CPU times: user 27.7 ms, sys: 10.1 ms, total: 37.8 ms
Wall time: 33.1 ms


In [30]:
%%time

Z = zVec(X,Y,tau)

CPU times: user 16.6 ms, sys: 5.23 ms, total: 21.8 ms
Wall time: 18.4 ms


In [31]:
def g(x):
    return GP_reg_intvl_2D(X, x, Z, tau)

In [32]:
import random

# Set a seed for reproducibility (optional)
random.seed(42)

# parameters for region of analysis
lower_bounds = [-0.001, -0.001]
upper_bounds = [90.0, 70.0]

# Generate twenty pairs of points
num_points = 20
point_set = [(random.uniform(lower_bounds[0],upper_bounds[0]), random.uniform(lower_bounds[1], upper_bounds[1])) for _ in range(num_points)]

# Display the generated points
for i, point in enumerate(point_set, 1):
    print(f"Point {i}: {point}")


Point 1: (57.548051288008004, 1.7497778763419085)
Point 2: (24.7519136825391, 15.623974881155743)
Point 3: (66.28214574597528, 47.36864081909122)
Point 4: (80.29605327300379, 6.08480522289176)
Point 5: (37.972385693494026, 2.0848351578843625)
Point 6: (19.676636370299107, 35.37437552252347)
Point 7: (2.38726380751741, 13.917834385716082)
Point 8: (58.48924928459488, 38.14544858370578)
Point 9: (19.838876424284745, 41.248187136997494)
Point 10: (72.84855053146107, 0.4539196762239493)
Point 11: (72.52353848420454, 48.869455788570875)
Point 12: (30.62188673713579, 10.882720466324521)
Point 13: (86.14913371168251, 23.560954752428987)
Point 14: (8.346218650056693, 6.7692430947193145)
Point 15: (76.27434046563772, 42.26042592171375)
Point 16: (72.64135172296749, 51.08095480035395)
Point 17: (48.26006445901453, 68.11807659431992)
Point 18: (34.06747248312902, 38.64239622975717)
Point 19: (74.6462491874338, 43.296001185249594)
Point 20: (77.55348273487026, 40.41422752011861)


In [33]:
for x in point_set:
    print('GP reg: %s' %f(x))
    print('GP interval: %s' %g(x))
    print('**************************************')


GP reg: [ 3.11929279 40.31468598]
GP interval: [interval([3.1192927884853754, 3.1192927884864914]), interval([40.314685979856826, 40.314685979863604])]
**************************************
GP reg: [15.09987151 17.33411459]
GP interval: [interval([15.099871506897031, 15.099871506911972]), interval([17.334114589381702, 17.334114589385802])]
**************************************
GP reg: [2.81857628e-02 4.63867769e+01]
GP interval: [interval([0.028185762779583585, 0.028185762780176583]), interval([46.38677685831422, 46.386776858322])]
**************************************
GP reg: [ 0.30405267 56.21273587]
GP interval: [interval([0.30405267116928836, 0.3040526711700797]), interval([56.21273587238129, 56.21273587239807])]
**************************************
GP reg: [14.20504423 26.31172842]
GP interval: [interval([14.205044229924173, 14.20504422993631]), interval([26.311728416762637, 26.311728416765433])]
**************************************
GP reg: [ 4.97363754 13.77321158]
GP inte

In [18]:
point_set[0]
    # norm_sq = (interval(a[0])-interval(b[0]))**2 + (interval(a[1])-interval(b[1]))**2


(57.548051288008004, 1.7497778763419085)

In [25]:
asdf = point_set[0]
qwer = point_set[1]
norm_sq_list = [(asdf[i]-qwer[i])**2 for i in range(len(asdf))]

In [26]:
norm_sq_list

[1075.5866418368514, 192.49334252838514]